In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
pip install split-folders


In [ ]:
# import splitfolders

In [ ]:
# input_folder="/gdrive/MyDrive/malaria_para_thesis/cell_images_input"
# output="/gdrive/MyDrive/malaria_para_thesis/processed_dataset"
#splitfolders.ratio(input_folder,output,seed=42,ratio=(.8, 0.2,.0))

In [ ]:
import numpy as np

#Set the `numpy` pseudo-random generator at a fixed value
#This helps with repeatable results everytime you run the code.
np.random.seed(1000)

import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import keras

os.environ['KERAS_BACKEND'] = 'tensorflow' # Added to set the backend as Tensorflow
#We can also set it to Theano if we want.


In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPool2D, Dense, Flatten,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
from glob import glob


import tensorflow as tf
import keras

In [ ]:
batch_size_ = 64
img_height,img_width = (64,64)


train_data_dir=r"/gdrive/MyDrive/malaria_para_thesis/processed_dataset/train"
valid_data_dir=r"/gdrive/MyDrive/malaria_para_thesis/processed_dataset/val"
#test_data_dir=r"flowers_dataset/processed_dataset/test"

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    # shear_range=0.2,
    #zoom_range=0.3,
    #horizontal_flip=True,
    #width_shift_range=0.2,
   # height_shift_range=0.2,
    #rotation_range=30
    )

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(64,64),
    batch_size=batch_size_)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(64,64),
    batch_size=batch_size_)


Found 22077 images belonging to 2 classes.
Found 5519 images belonging to 2 classes.


In [ ]:
len(train_generator)

690

In [ ]:
x,y=val_generator.next()
x.shape,y.shape

((32, 256, 256, 3), (32, 2))

In [ ]:
lr_ = 1e-3
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
SIZE = 64

In [ ]:
# model = tf.keras.applications.ResNet50(
#     include_top=False,
#     weights="imagenet",
#     input_shape=(256, 256,3),
#     pooling=None,
#     classes=2,
# )
# # densenet 121, 169, 201
# x = GlobalAveragePooling2D()(model.output)
# x = Dense(1024, 'relu')(x)
# x = Dense(256, 'relu')(x)
# x = Dense(2, 'softmax')(x)

# model = Model(model.input, x)
# model.summary()

# model.compile(tf.keras.optimizers.Adam(learning_rate=lr_),
#                   'categorical_crossentropy',
#                   ['accuracy'])


#Apply CNN
# ### Build the model

#############################################################
###2 conv and pool layers. with some normalization and drops in between.

INPUT_SHAPE = (SIZE, SIZE, 3)   #change to (SIZE, SIZE, 3)
inp = keras.layers.Input(shape=INPUT_SHAPE)

conv1 = keras.layers.Conv2D(32, kernel_size=(3, 3),
                               activation='relu', padding='same')(inp)
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
norm1 = keras.layers.BatchNormalization(axis = -1)(pool1)
drop1 = keras.layers.Dropout(rate=0.2)(norm1)
conv2 = keras.layers.Conv2D(32, kernel_size=(3, 3),
                               activation='relu', padding='same')(drop1)
pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
norm2 = keras.layers.BatchNormalization(axis = -1)(pool2)
drop2 = keras.layers.Dropout(rate=0.2)(norm2)

flat = keras.layers.Flatten()(drop2)  #Flatten the matrix to get it ready for dense.

hidden1 = keras.layers.Dense(512, activation='relu')(flat)
norm3 = keras.layers.BatchNormalization(axis = -1)(hidden1)
drop3 = keras.layers.Dropout(rate=0.2)(norm3)
hidden2 = keras.layers.Dense(256, activation='relu')(drop3)
norm4 = keras.layers.BatchNormalization(axis = -1)(hidden2)
drop4 = keras.layers.Dropout(rate=0.2)(norm4)

out = keras.layers.Dense(2, activation='sigmoid')(drop4)   #units=1 gives error

model = keras.Model(inputs=inp, outputs=out)
model.compile(optimizer='adam',
                loss='categorical_crossentropy',   #Check between binary_crossentropy and categorical_crossentropy
                metrics=['accuracy'])
print(model.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248  

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
annelear = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, verbose=1,
                             mode='auto', min_lr=1e-8)

path_to_save = 'gdrive/MyDrive/malaria_para_thesis/resnet_50_epo20.h5'

#'/content/PBF0_d212.h5'
#'/content/drive/MyDrive/Terms/Thesis/WCE DATA/Trainings/d169_fold'+str(fold)+'.h5'   ---> Random Fold

save_W = ModelCheckpoint(path_to_save,
                          monitor='val_accuracy',mode ='max',verbose=1,
                          save_best_only=True, save_weights_only=True)

history = model.fit(train_generator,
                        batch_size=batch_size_,
                        epochs=10, verbose=1,
                        validation_data=(val_generator),
                        # shuffle=True,
                        callbacks=[annelear, save_W],)

In [ ]:
# ## Accuracy calculation
#
# I'll now calculate the accuracy on the test data.

#print("Test_Accuracy: {:.2f}%".format(model.evaluate(np.array(X_test), np.array(y_test))[1]*100))




f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('CNN Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

max_epoch = len(history.history['accuracy'])+1
epoch_list = list(range(1,max_epoch))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(1, max_epoch, 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(1, max_epoch, 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
#model_acc = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)[1]
#print("Test Accuracy: {:.3f}%".format(model_acc * 100))
test_loss,test_acc=model.evaluate(val_generator,verbose=1)
print('\nTest_accuracy',test_acc)

In [ ]:
import tensorflow as tf

from keras.models import load_model

#model.save('/gdrive/MyDrive/malaria_para_thesis/try1_30epoch.h5')

In [ ]:
# model=tf.keras.models.load_model("/gdrive/MyDrive/malaria_para_thesis/weights-improvement.h5")
# score = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
# print('Test accuracy:', score[1])                                                                        for comment ctrl+/

test_loss,test_acc=model.evaluate(val_generator,verbose=1)
print('\nTest_accuracy',test_acc)

In [ ]:
weight_ = '/gdrive/MyDrive/flower/resnet_50_epo10.h5'
model.load_weights(weight_)

In [ ]:

# from tqdm import tqdm

In [ ]:
# train_generator.reset()
# X_train, y_train = next(train_generator)
# for i in tqdm(range(int(len(train_generator)/batch_size_)-1)): #1st batch is already fetched before the for loop.
#   img, label = next(train_generator)
#   X_train = np.append(X_train, img, axis=0 )
#   y_train = np.append(y_train, label, axis=0)
# print(X_train.shape, y_train.shape)

100%|██████████| 20/20 [00:17<00:00,  1.12it/s]

(672, 256, 256, 3) (672, 2)


In [ ]:
from sklearn.metrics import classification_report
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from sklearn.metrics import accuracy_score, confusion_matrix
from seaborn import heatmap
from matplotlib import pyplot as plt


In [ ]:
# #models=tf.keras.models.load_model("dataset_2\ResNet50_xray224.h5")
# preds = np.argmax(model.predict(np.array(X_test)), axis=1)
# acc = accuracy_score(y_testt, np.round(preds))*100
# cm = confusion_matrix(y_testt, np.round(preds))
# cm_norm = confusion_matrix(y_testt, np.round(preds), normalize='true')
# #tn, fp, fn, tp = cm.ravel()

# print('CONFUSION MATRIX ------------------')

# ax = heatmap(cm, cmap='Blues', linecolor='lightblue',linewidths=.5,annot=True,annot_kws={'size': 12,"weight": "bold"}, xticklabels=['parasitized', 'uninfected'], yticklabels= ['parasitized', 'uninfected'], square=True, fmt='d')

# ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 12)
# ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 12,rotation=0)

# # plt.savefig(confusion_matrix_save_path+model_name+'.png',dpi=500)
# plt.show()

# ax = heatmap(cm_norm, cmap='Blues', annot=True,linecolor='lightblue',linewidths=.5,annot_kws={'size': 12,"weight": "bold"}, xticklabels=['parasitized', 'uninfected'], yticklabels=['parasitized', 'uninfected'], square=True, fmt='.2f')

# ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 12)
# ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 12,rotation=0)

# # plt.savefig(confusion_matrix_save_path+model_name+'_normalized.png',dpi=500)
# plt.show()


In [ ]:
#print(classification_report(y_testt, preds))

In [ ]:
import pandas as pd
import seaborn as sn
import tensorflow as tf
model=tf.keras.models.load_model("ResNet50_flower.h5")
filenames=test_generator.filenames
nb_samples=len(val_generator)
y_prob=[]
y_act=[]
val_generator.reset()

for _ in range(nb_samples):
    X_test,Y_test=val_generator.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)

predicted_class=[list(train_generator.class_indices.keys())[i.argmax()]for i in y_prob]
actual_class=[list(train_generator.class_indices.keys())[i.argmax()]for i in y_act]
out_df=pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns=['predicted_class','actual_class'])



In [ ]:
confusion_matrix=pd.crosstab(out_df['actual_class'],out_df['predicted_class'],rownames=['Actual'],colnames=['Predicted'])
sn.heatmap(confusion_matrix,cmap='Blues',annot=True,fmt='d')